In [ ]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/train_preprocessed.csv')

In [ ]:
val_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/val_preprocessed.csv')

In [ ]:
train_X = train_df['preprocessed_text'].to_list()
train_Y = (train_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [ ]:
val_X = val_df['preprocessed_text'].to_list()
val_Y = (val_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [ ]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_X)

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        mask_zero=True),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, None, 128)         1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,321,473
Trainable params: 1,321,473
Non-trainable params: 0
____________________________________________

In [ ]:
history = model.fit(x = train_X, y = train_Y, epochs=10,
                    validation_data=(val_X, val_Y),
                    validation_steps=100)

Epoch 1/10
331/331 [==============================] - ETA: 0s - loss: 0.6474 - accuracy: 0.6710

331/331 [==============================] - 25s 61ms/step - loss: 0.6474 - accuracy: 0.6710 - val_loss: 0.6286 - val_accuracy: 0.6545
Epoch 2/10
331/331 [==============================] - 17s 53ms/step - loss: 0.5838 - accuracy: 0.6712
Epoch 3/10
331/331 [==============================] - 22s 65ms/step - loss: 0.4594 - accuracy: 0.7685
Epoch 4/10
331/331 [==============================] - 18s 53ms/step - loss: 0.3369 - accuracy: 0.8592
Epoch 5/10
331/331 [==============================] - 18s 54ms/step - loss: 0.2671 - accuracy: 0.8934
Epoch 6/10
331/331 [==============================] - 17s 53ms/step - loss: 0.2160 - accuracy: 0.9176
Epoch 7/10
331/331 [==============================] - 17s 52ms/step - loss: 0.1812 - accuracy: 0.9339
Epoch 8/10
331/331 [==============================] - 18s 53ms/step - loss: 0.1529 - accuracy: 0.9482
Epoch 9/10
331/331 [==============================] - 17s 53ms/step - loss: 0.1335 - accuracy: 0.9542
Epoch 10/10
331/331 [==============================

In [ ]:
train_pred = model.predict(train_X)
val_pred = model.predict(val_X)

83/83 [==============================] - 2s 11ms/step


In [ ]:
train_pred = np.where(train_pred > 0.5, 1, 0)
val_pred = np.where(val_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

def computeAllScores(y_pred_train, y_pred_dev, train_labels, dev_labels):
    print("Accuracy Train: ", accuracy_score(train_labels, y_pred_train))
    print("Accuracy Dev: ", accuracy_score(dev_labels, y_pred_dev))
    print("Weighted F1 Train: ", f1_score(train_labels, y_pred_train, average='weighted'))
    print("Weighted F1 Dev: ", f1_score(dev_labels, y_pred_dev, average='weighted'))
    print("Macro F1 Train: ", f1_score(train_labels, y_pred_train, average='macro'))
    print("Macro F1 Dev: ", f1_score(dev_labels, y_pred_dev, average='macro'))
    print("Micro F1 Train: ", f1_score(train_labels, y_pred_train, average='micro'))
    print("Micro F1 Dev: ", f1_score(dev_labels, y_pred_dev, average='micro'))
    print("Weighted Recall Train: ", recall_score(train_labels, y_pred_train, average='weighted'))
    print("Weighted Recall Dev: ", recall_score(dev_labels, y_pred_dev, average='weighted'))
    print("Macro Recall Train: ", recall_score(train_labels, y_pred_train, average='macro'))
    print("Macro Recall Dev: ", recall_score(dev_labels, y_pred_dev, average='macro'))
    print("Micro Recall Train: ", recall_score(train_labels, y_pred_train, average='micro'))
    print("Micro Recall Dev: ", recall_score(dev_labels, y_pred_dev, average='micro'))
    # Confusion Matrix
    print("Confusion Matrix Train: ")
    print(confusion_matrix(train_labels, y_pred_train))
    print("Confusion Matrix Dev: ")
    print(confusion_matrix(dev_labels, y_pred_dev))

In [ ]:
computeAllScores(train_pred, val_pred, train_Y, val_Y)

Accuracy Train:  0.9705438066465257
Accuracy Dev:  0.7246978851963746
Weighted F1 Train:  0.9703997195562272
Weighted F1 Dev:  0.7176376889199447
Macro F1 Train:  0.9663225051729393
Macro F1 Dev:  0.681315421283972
Micro F1 Train:  0.9705438066465257
Micro F1 Dev:  0.7246978851963745
Weighted Recall Train:  0.9705438066465257
Weighted Recall Dev:  0.7246978851963746
Macro Recall Train:  0.9619636630403997
Macro Recall Dev:  0.6751496977665945
Micro Recall Train:  0.9705438066465257
Micro Recall Dev:  0.7246978851963746
Confusion Matrix Train: 
[[7015   92]
 [ 220 3265]]
Confusion Matrix Dev: 
[[1448  285]
 [ 444  471]]


In [ ]:
val_pred = np.where(val_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import f1_score

f1_score(val_Y, val_pred, average='macro')

0.6926490155178788

In [ ]:
len(train_X)/len(val_X)

4.0

In [ ]:
len(val_X)

2648